In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import datetime
import geocoder
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
pd.set_option('max_colwidth',200)
%matplotlib inline

In [29]:
import requests
import json
import urllib
from urllib.request import urlopen

def getGeoForAddress(address):
    add = urllib.parse.quote(address)
    addressUrl = "https://maps.googleapis.com/maps/api/geocode/json?address=" + add + '&key='
    res = requests.get(addressUrl)
    js = json.loads(res.text)
    if js.get('results'):
        lat = js.get('results')[0]['geometry']['location']['lat']
        lng = js.get('results')[0]['geometry']['location']['lng']
    else:
        lat = ''
        lng = ''
    return [lat, lng]

In [143]:
import os
import pandas as pd
import numpy as np
from itertools import chain 
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt

In [2]:
firm_2004 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2004.csv',low_memory=False)
firm_2004['鄉鎮碼'] = firm_2004['行政区代码'].astype(str).map(lambda x:x[0:9])
firm_2004['縣碼'] = firm_2004['行政区代码'].astype(str).map(lambda x:x[0:6])

In [2]:
firm_2006 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2006.csv',low_memory=False)
firm_2006['鄉鎮碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:9])
firm_2006['縣碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:6])
firm_2005 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2005.csv',low_memory=False)
firm_2005['鄉鎮碼'] = firm_2005['區域代碼'].astype(str).map(lambda x:x[0:9])
firm_2008 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2008.csv',low_memory=False)
firm_2008['鄉鎮碼'] = firm_2008['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2010 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2010.csv',low_memory=False)
firm_2010['鄉鎮碼'] = firm_2010['行政区划代码'].astype(str).map(lambda x:x[0:9])

In [129]:
firm_2007 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2007.csv',low_memory=False)
firm_2007['鄉鎮碼'] = firm_2007['區域代碼'].astype(str).map(lambda x:x[0:9])

# 周邊高等學校數量(不用)

In [33]:
university1 = pd.read_excel(r'C:\Users\User\Dropbox\paper\科技園區\China\2014_school1.xls',skiprows=2)
university2 = pd.read_excel(r'C:\Users\User\Dropbox\paper\科技園區\China\2014_school2.xls',skiprows=2)

In [40]:
university2['學校'] = university2.apply(lambda x:x['所在地']+x['学校名称'],axis=1)
university2.dropna(subset=['學校'],inplace=True)
university2['學校'] = university2['學校'].map(lambda x:re.sub(r"\s+","",x))
university1['學校'] = university1['学校名称']
university1.dropna(subset=['學校'],inplace=True)
university1['學校'] = university1['學校'].map(lambda x:re.sub(r"\s+","",x))
university = pd.concat([university1[['學校','主管部门']],university2[['學校','主管部门']]],ignore_index=True,sort=False)
university.loc[:,'lat'] = ''
university.loc[:,'lnt'] = ''
university['鄉鎮碼'] = ''

In [56]:
for i in university[university['lat']==''].index:
    address = university.loc[i,'學校']
    g = getGeoForAddress(address)
    university.loc[i,'lat'] = g[0]
    university.loc[i,'lnt'] = g[1]

In [226]:
university.to_excel(r'E:\paper\大學位置.xlsx')

In [225]:
university

,學校,主管部门,lat,lnt,鄉鎮碼
0,北京市海淀区职工大学,北京市,39.9873,116.345,110108010
1,北京市东城区职工业余大学,北京市,39.9319,116.436,110101009
2,北京市崇文区职工大学,北京市,39.8857,116.442,110229101
3,北京宣武红旗业余大学,北京市,39.8817,116.337,110102020
4,北京市石景山区业余大学,北京市,39.9172,116.2,110107003
...,...,...,...,...,...
2537,乌鲁木齐市新疆师范高等专科学校,新疆维吾尔自治区,43.8039,87.6178,650102012
2538,乌鲁木齐市新疆铁道职业技术学院,新疆维吾尔自治区,43.8372,87.5962,650103004
2539,乌鲁木齐市新疆生产建设兵团兴新职业技术学院,新疆生产建设兵团,43.8256,87.6168,650104008
2540,哈密市哈密职业技术学院,新疆维吾尔自治区,42.8371,93.4934,


In [146]:
os.chdir(r'E:\paper\名稱merge')
files = os.listdir(os.getcwd())

In [153]:
from geopy.distance import geodesic
def how_far(a,b):
    try:
        return geodesic(a,b).kilometers
    except:
        return np.nan

In [224]:
os.chdir(r'E:\paper\名稱merge')
files = os.listdir(os.getcwd())
for i in files:
    firm = pd.read_excel(i,index_col=0)
    region = firm.loc[0,'address'].split('省')[0]
    firm.fillna('',inplace=True)
    firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
    firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])
    for k in university[(university['主管部门'].str.contains(region))&(university['lat']!='')&(university['鄉鎮碼']=='')].index:
        target = (university.loc[k,'lat'],university.loc[k,'lnt'])
        firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
        university.loc[k,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [228]:
firm = pd.read_excel(r'E:\paper\名稱merge\重慶_結果.xlsx',index_col=0)
region = '重庆'
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])
for k in university[(university['主管部门'].str.contains(region))&(university['lat']!='')].index:
    target = (university.loc[k,'lat'],university.loc[k,'lnt'])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[k,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [229]:
firm = pd.read_excel(r'E:\paper\名稱merge\內蒙古_結果.xlsx',index_col=0)
region = '内蒙古'
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])
for k in university[(university['主管部门'].str.contains(region))&(university['lat']!='')].index:
    target = (university.loc[k,'lat'],university.loc[k,'lnt'])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[k,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [230]:
firm = pd.read_excel(r'E:\paper\名稱merge\寧夏_結果.xlsx',index_col=0)
region = '宁夏'
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])
for k in university[(university['主管部门'].str.contains(region))&(university['lat']!='')].index:
    target = (university.loc[k,'lat'],university.loc[k,'lnt'])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[k,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [231]:
firm = pd.read_excel(r'E:\paper\名稱merge\新疆_結果.xlsx',index_col=0)
region = '新疆'
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])
for k in university[(university['主管部门'].str.contains(region))&(university['lat']!='')].index:
    target = (university.loc[k,'lat'],university.loc[k,'lnt'])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[k,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [246]:
firm = pd.read_excel(r'E:\paper\名稱merge\北京_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['中央党校继续教育学院','国家开放大学（中央广播电视大学）','民航管理干部学院','国家法官学院','首都联合职工大学','国家检察官学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [248]:
firm = pd.read_excel(r'E:\paper\名稱merge\河北_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['三河市华北科技学院','廊坊市中国人民武装警察部队学院','三河市防灾科技学院','保定市中央司法警官学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [249]:
firm = pd.read_excel(r'E:\paper\名稱merge\遼寧_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['大连市大连理工大学','沈阳市东北大学','大连市大连海事大学','沈阳市中国刑事警察学院','大连市大连民族学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [250]:
firm = pd.read_excel(r'E:\paper\名稱merge\吉林_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['长春市东北师范大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [251]:
firm = pd.read_excel(r'E:\paper\名稱merge\黑龍江_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['哈尔滨市哈尔滨工业大学','哈尔滨市哈尔滨工程大学','哈尔滨市东北林业大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [252]:
firm = pd.read_excel(r'E:\paper\名稱merge\江蘇_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['南京市南京大学','南京市东南大学','南京市南京航空航天大学','南京市南京理工大学','徐州市中国矿业大学','南京市河海大学','无锡市江南大学'
          ,'南京市南京农业大学','南京市中国药科大学','南京市南京森林警察学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [254]:
firm = pd.read_excel(r'E:\paper\名稱merge\浙江_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['宁波市公安海警学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [255]:
firm = pd.read_excel(r'E:\paper\名稱merge\安徽_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['合肥市合肥工业大学','合肥市中国科学技术大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [256]:
firm = pd.read_excel(r'E:\paper\名稱merge\福建_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['厦门市厦门大学','泉州市华侨大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [257]:
firm = pd.read_excel(r'E:\paper\名稱merge\山東_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['青岛市中国海洋大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [258]:
firm = pd.read_excel(r'E:\paper\名稱merge\河南_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['郑州市铁道警察学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [259]:
firm = pd.read_excel(r'E:\paper\名稱merge\湖北_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['武汉市武汉大学','武汉市华中科技大学','武汉市中国地质大学','武汉市武汉理工大学','武汉市华中农业大学','武汉市华中师范大学','武汉市中南财经政法大学','武汉市中南民族大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [260]:
firm = pd.read_excel(r'E:\paper\名稱merge\廣東_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['广州市中山大学','广州市华南理工大学','广州市暨南大学','广州市广州民航职业技术学院']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [261]:
firm = pd.read_excel(r'E:\paper\名稱merge\重慶_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['重庆市重庆大学','重庆市西南大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [262]:
firm = pd.read_excel(r'E:\paper\名稱merge\四川_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['成都市西南交通大学','成都市电子科技大学','成都市西南财经大学','广汉市中国民用航空飞行学院','成都市西南民族大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [263]:
firm = pd.read_excel(r'E:\paper\名稱merge\陝西_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['西安市西安交通大学','咸阳市西北农林科技大学','西安市长安大学','西安市西安电子科技大学','西安市西北工业大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [264]:
firm = pd.read_excel(r'E:\paper\名稱merge\甘肅_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['兰州市兰州大学','兰州市西北民族大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [265]:
firm = pd.read_excel(r'E:\paper\名稱merge\寧夏_結果.xlsx',index_col=0)
firm.fillna('',inplace=True)
firm = firm[(firm['區域代碼']!='')&(firm['lat']!='')]
firm['鄉鎮碼'] = firm['區域代碼'].astype(str).map(lambda x:x[0:9])

school = ['银川市北方民族大学']
for i in school:
    target = (university.loc[university[university['學校']==i].index,'lat'].values[0],university.loc[university[university['學校']==i].index,'lnt'].values[0])
    firm['target_distance'] = firm.apply(lambda x:how_far(target,(x['lat'],x['lnt'])),axis=1)
    university.loc[university[university['學校']==i].index,'鄉鎮碼'] = firm.loc[firm[firm['target_distance']==firm['target_distance'].min()].index[0],'鄉鎮碼']

In [266]:
university.to_excel(r'E:\paper\大學鄉鎮碼.xlsx')

# 員工教育水準

In [4]:
#9碼
edu_9 = firm_2004.groupby('鄉鎮碼')[['从业人员总','研究生','本科','专科']].sum()
edu_9['高學歷'] = edu_9.apply(lambda x:x['研究生']+x['本科']+x['专科'],axis=1)
edu_9['比例'] = edu_9.apply(lambda x:x['高學歷']/x['从业人员总'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.


In [5]:
#6碼
edu_6 = firm_2004.groupby('縣碼')[['从业人员总','研究生','本科','专科']].sum()
edu_6['高學歷'] = edu_6.apply(lambda x:x['研究生']+x['本科']+x['专科'],axis=1)
edu_6['比例'] = edu_6.apply(lambda x:x['高學歷']/x['从业人员总'],axis=1)

In [6]:
edu_6

,从业人员总,研究生,本科,专科,高學歷,比例
縣碼,,,,,,
110101,10876,225,1611,1503,3339,0.307006
110102,45122,1535,6811,8479,16825,0.372878
110103,11771,125,1105,1919,3149,0.267522
110104,28561,382,3173,5415,8970,0.314065
110105,161702,2147,16652,23713,42512,0.262903
...,...,...,...,...,...,...
654326,135,1,2,26,29,0.214815
659001,23748,48,1175,5616,6839,0.287982
659002,150,0,4,26,30,0.200000


# Capital/Labor ratio

In [7]:
#9碼
clratio_9 = firm_2004.groupby('鄉鎮碼')[['固定资产合','从业人员总']].sum()
clratio_9['capital labor ratio'] = clratio_9.apply(lambda x:x['固定资产合']/x['从业人员总'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#6碼
clratio_6 = firm_2004.groupby('縣碼')[['固定资产合','从业人员总']].sum()
clratio_6['capital labor ratio'] = clratio_6.apply(lambda x:x['固定资产合']/x['从业人员总'],axis=1)

In [9]:
clratio_6

,固定资产合,从业人员总,capital labor ratio
縣碼,,,
110101,1701040,10876,156.403089
110102,71284883,45122,1579.825429
110103,10166904,11771,863.724747
110104,28694768,28561,1004.683590
110105,47551447,161702,294.068391
...,...,...,...
654326,3540,135,26.222222
659001,4579948,23748,192.856156
659002,57569,150,383.793333


# 平均薪資

In [10]:
#9碼
salary_9 = firm_2004.groupby('鄉鎮碼')[['从业人员总','应付工资总']].sum()
salary_9['平均薪資'] = salary_9.apply(lambda x:x['应付工资总']/x['从业人员总'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#6碼
salary_6 = firm_2004.groupby('縣碼')[['从业人员总','应付工资总']].sum()
salary_6['平均薪資'] = salary_6.apply(lambda x:x['应付工资总']/x['从业人员总'],axis=1)

In [12]:
salary_6

,从业人员总,应付工资总,平均薪資
縣碼,,,
110101,10876,407406,37.459176
110102,45122,1731581,38.375537
110103,11771,368113,31.272874
110104,28561,1908534,66.823080
110105,161702,4950609,30.615632
...,...,...,...
654326,135,1488,11.022222
659001,23748,269680,11.355904
659002,150,1155,7.700000


# R&D intensity(不用)

In [16]:
#9碼
randd = firm_2006.groupby('鄉鎮碼')[['研究开发费','工业总产值']].sum()
randd['R&D'] = randd.apply(lambda x:x['研究开发费']/x['工业总产值'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in longlong_scalars
  


In [11]:
#6碼
randd = firm_2006.groupby('縣碼')[['研究开发费','工业总产值']].sum()
randd['R&D'] = randd.apply(lambda x:x['研究开发费']/x['工业总产值'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
randd

,研究开发费,工业总产值,R&D
縣碼,,,
110101,114876,5305204,0.021653
110102,289825,54093012,0.005358
110103,69087,3706661,0.018639
110104,326404,30701681,0.010631
110105,1001736,67688615,0.014799
...,...,...,...
654326,0,12146,0.000000
659001,1900,6000416,0.000317
659002,0,334873,0.000000


# R&D intensity(OECD)

In [13]:
aircraft = [3761,3762,3769]
phar = [2710,2720,2730,2740,2750,2760,2770]
commu = [4011,4012,4019,4013,4014,4031,4032,4039,4071,4072,4051,4052,4053,4059,4061,4062]
office = [4041,4042,4043,4154,4155,2665,6211,6212]
precise = [3681,3682,3683,3684,3685,3686,3689,4111,4112,4113,4114,4115,4119,4121,4122,4123,4124,4125,4126,4127,4128,4129,4141,4190]

elec = ['39','40','41']
vehicle = ['37']
chem_phar = ['26','27']
equip = ['35','36']

rubber = ['28','29','30']
fuel = ['25']
nonmetal = ['31']
metal = ['32','33','34']

manu = ['42','43']
wood = ['20','21','22','23','24']
food = ['13','14','15','16']
textile = ['17','18','19']

In [14]:
tech_firm = firm_2004
tech_firm['產業二位碼'] = tech_firm['行业代码'].astype(str).map(lambda x:x[:-2])

In [15]:
def rnd(a,b):
    if a in aircraft:
        return 13.3
    elif a in phar:
        return 10.5
    elif a in commu:
        return 9.2
    elif a in office:
        return 8
    elif a in precise:
        return 7.7
    elif b in elec:
        return 3.9
    elif b in vehicle:
        return 3.5
    elif b in chem_phar:
        return 3.1
    elif b in equip:
        return 2.1
    elif b in rubber:
        return 0.9
    elif b in fuel:
        return 0.9
    elif b in nonmetal:
        return 0.9
    elif b in metal:
        return 0.6
    elif b in manu:
        return 0.5
    elif b in wood:
        return 0.3
    elif b in food:
        return 0.3
    elif b in textile:
        return 0.3
    else:
        return 0.3

In [16]:
tech_firm['R&D OECD'] = tech_firm.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

In [17]:
#9碼
tech_per_9 = tech_firm.groupby('鄉鎮碼')['R&D OECD'].mean()

In [18]:
#6碼
tech_per_6 = tech_firm.groupby('縣碼')['R&D OECD'].mean()

In [19]:
tech_per_6

縣碼
110101    3.502151
110102    2.948276
110103    2.396721
110104    2.080198
110105    2.765900
            ...   
654326    0.900000
659001    0.711538
659002    0.300000
659003    0.660000
659004    1.271429
Name: R&D OECD, Length: 2849, dtype: float64

# 大學數量(不用)

In [19]:
uni = pd.read_excel(r'E:\paper\大學鄉鎮碼.xlsx',index_col=0)
uni.fillna('0',inplace=True)
uni['鄉鎮碼'] = uni['鄉鎮碼'].astype(str).map(lambda x:x[0:9])
uni['縣碼'] = uni['鄉鎮碼'].astype(str).map(lambda x:x[0:6])

In [39]:
#9碼
uni_count = uni.groupby('鄉鎮碼')['學校'].count()
uni_count.drop('0',inplace=True)

In [20]:
#6碼
uni_count = uni.groupby('縣碼')['學校'].count()
uni_count.drop('0',inplace=True)

In [21]:
uni_count

縣碼
110101     5
110102     9
110105    16
110106     7
110107     3
          ..
653101     1
653201     3
654002     3
659001     3
659002     1
Name: 學校, Length: 825, dtype: int64

# 彙整(9碼)

In [20]:
edu_9

,从业人员总,研究生,本科,专科,高學歷,比例
鄉鎮碼,,,,,,
110101001,1075,141,602,60,803,0.746977
110101002,1141,23,119,159,301,0.263804
110101003,275,4,29,68,101,0.367273
110101004,548,3,28,104,135,0.246350
110101005,1338,7,65,107,179,0.133782
...,...,...,...,...,...,...
659004100,469,1,13,89,103,0.219616
659004101,1597,0,26,189,215,0.134627
659004102,778,0,3,78,81,0.104113


In [21]:
clratio_9

,固定资产合,从业人员总,capital labor ratio
鄉鎮碼,,,
110101001,43258,1075,40.240000
110101002,295047,1141,258.586328
110101003,4091,275,14.876364
110101004,24641,548,44.965328
110101005,37559,1338,28.071001
...,...,...,...
659004100,39516,469,84.255864
659004101,117305,1597,73.453350
659004102,17396,778,22.359897


In [22]:
tech_per_9

鄉鎮碼
110101001    4.112500
110101002    3.122222
110101003    4.771429
110101004    5.687500
110101005    3.944444
               ...   
659004100    1.980000
659004101    0.450000
659004102    0.300000
659004400    2.100000
659004401    3.100000
Name: R&D OECD, Length: 24601, dtype: float64

In [23]:
salary_9

,从业人员总,应付工资总,平均薪資
鄉鎮碼,,,
110101001,1075,115185,107.148837
110101002,1141,27622,24.208589
110101003,275,6281,22.840000
110101004,548,8355,15.246350
110101005,1338,25351,18.946936
...,...,...,...
659004100,469,3620,7.718550
659004101,1597,14031,8.785848
659004102,778,1720,2.210797


In [24]:
control_var_9 = pd.merge(edu_9['比例'],salary_9['平均薪資'], left_index=True, right_index=True, how='outer')
control_var_9 = pd.merge(control_var_9,clratio_9['capital labor ratio'], left_index=True, right_index=True, how='outer')
control_var_9 = pd.merge(control_var_9,tech_per_9, left_index=True, right_index=True, how='outer')

In [25]:
print(control_var_9.shape)

(24601, 4)


In [27]:
control_var_9 = control_var_9[(control_var_9['比例']!=np.inf)&(control_var_9['比例']!=-np.inf)]
control_var_9 = control_var_9[(control_var_9['平均薪資']!=np.inf)&(control_var_9['平均薪資']!=-np.inf)]
control_var_9 = control_var_9[(control_var_9['capital labor ratio']!=np.inf)&(control_var_9['capital labor ratio']!=-np.inf)]

In [28]:
print(control_var_9.shape)

(24598, 4)


In [29]:
control_var_9

,比例,平均薪資,capital labor ratio,R&D OECD
鄉鎮碼,,,,
110101001,0.746977,107.148837,40.240000,4.112500
110101002,0.263804,24.208589,258.586328,3.122222
110101003,0.367273,22.840000,14.876364,4.771429
110101004,0.246350,15.246350,44.965328,5.687500
110101005,0.133782,18.946936,28.071001,3.944444
...,...,...,...,...
659004100,0.219616,7.718550,84.255864,1.980000
659004101,0.134627,8.785848,73.453350,0.450000
659004102,0.104113,2.210797,22.359897,0.300000


In [30]:
control_var_9.to_excel(r'E:\paper\baseline_9碼(2004)_new.xlsx')

# 彙整(6碼)

In [31]:
edu_6

,从业人员总,研究生,本科,专科,高學歷,比例
縣碼,,,,,,
110101,10876,225,1611,1503,3339,0.307006
110102,45122,1535,6811,8479,16825,0.372878
110103,11771,125,1105,1919,3149,0.267522
110104,28561,382,3173,5415,8970,0.314065
110105,161702,2147,16652,23713,42512,0.262903
...,...,...,...,...,...,...
654326,135,1,2,26,29,0.214815
659001,23748,48,1175,5616,6839,0.287982
659002,150,0,4,26,30,0.200000


In [32]:
clratio_6

,固定资产合,从业人员总,capital labor ratio
縣碼,,,
110101,1701040,10876,156.403089
110102,71284883,45122,1579.825429
110103,10166904,11771,863.724747
110104,28694768,28561,1004.683590
110105,47551447,161702,294.068391
...,...,...,...
654326,3540,135,26.222222
659001,4579948,23748,192.856156
659002,57569,150,383.793333


In [33]:
tech_per_6

縣碼
110101    3.502151
110102    2.948276
110103    2.396721
110104    2.080198
110105    2.765900
            ...   
654326    0.900000
659001    0.711538
659002    0.300000
659003    0.660000
659004    1.271429
Name: R&D OECD, Length: 2849, dtype: float64

In [34]:
salary_6

,从业人员总,应付工资总,平均薪資
縣碼,,,
110101,10876,407406,37.459176
110102,45122,1731581,38.375537
110103,11771,368113,31.272874
110104,28561,1908534,66.823080
110105,161702,4950609,30.615632
...,...,...,...
654326,135,1488,11.022222
659001,23748,269680,11.355904
659002,150,1155,7.700000


In [35]:
control_var_6 = pd.merge(edu_6['比例'],salary_6['平均薪資'], left_index=True, right_index=True, how='outer')
control_var_6 = pd.merge(control_var_6,clratio_6['capital labor ratio'], left_index=True, right_index=True, how='outer')
control_var_6 = pd.merge(control_var_6,tech_per_6, left_index=True, right_index=True, how='outer')

In [36]:
print(control_var_6.shape)

(2849, 4)


In [37]:
control_var_6 = control_var_6[(control_var_6['比例']!=np.inf)&(control_var_6['比例']!=-np.inf)]
control_var_6 = control_var_6[(control_var_6['平均薪資']!=np.inf)&(control_var_6['平均薪資']!=-np.inf)]
control_var_6 = control_var_6[(control_var_6['capital labor ratio']!=np.inf)&(control_var_6['capital labor ratio']!=-np.inf)]

In [39]:
print(control_var_6.shape)

(2849, 4)


In [40]:
control_var_6

,比例,平均薪資,capital labor ratio,R&D OECD
縣碼,,,,
110101,0.307006,37.459176,156.403089,3.502151
110102,0.372878,38.375537,1579.825429,2.948276
110103,0.267522,31.272874,863.724747,2.396721
110104,0.314065,66.823080,1004.683590,2.080198
110105,0.262903,30.615632,294.068391,2.765900
...,...,...,...,...
654326,0.214815,11.022222,26.222222,0.900000
659001,0.287982,11.355904,192.856156,0.711538
659002,0.200000,7.700000,383.793333,0.300000


In [41]:
control_var_6.to_excel(r'E:\paper\baseline_6碼(2004)_new.xlsx')